In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('../data/survey_data_before_pre_processing.csv')

In [5]:
df.head()

,wave,year,weight_wave,weight_aggregate,happening,cause_original,cause_recoded,sci_consensus,worry,harm_personally,...,house_size,house_ages0to1,house_ages2to5,house_ages6to12,house_ages13to17,house_ages18plus,house_type,house_own,age_category_35_54,age_category_55_plus
0,2008 Nov,2008,0.54,0.294402,Yes,Caused mostly by human activities,Caused mostly by human activities,Most scientists think global warming is happening,Somewhat worried,Only a little,...,3.0,0.0,0.0,0.0,0.0,3.0,One-family house detached from any other house,Owned by you or someone in your household,0,1
1,2008 Nov,2008,0.85,0.463411,Don't know,Caused mostly by human activities,Caused mostly by human activities,Don't know enough to say,Not very worried,Only a little,...,2.0,0.0,0.0,0.0,0.0,2.0,Mobile home,Rented,1,0
2,2008 Nov,2008,0.49,0.267143,Don't know,Caused mostly by natural changes in the enviro...,Caused mostly by natural changes in the enviro...,There is a lot of disagreement among scientist...,Not at all worried,Not at all,...,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household,1,0
3,2008 Nov,2008,0.29,0.158105,Yes,Caused mostly by natural changes in the enviro...,Caused mostly by natural changes in the enviro...,Most scientists think global warming is happening,Somewhat worried,Only a little,...,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household,0,1
4,2008 Nov,2008,1.29,0.703294,Yes,Caused mostly by human activities,Caused mostly by human activities,There is a lot of disagreement among scientist...,Somewhat worried,Don't know,...,2.0,0.0,0.0,0.0,0.0,2.0,One-family house detached from any other house,Owned by you or someone in your household,0,0
